This notebook is intended to solve the "general" problem of steering in the linac by solving for correctors which give the optimal emittance and steering at some final checkpoint

Practically, the purpose here is to give the final refinement to a one-to-one steering solution produced by, e.g. "2025-06-12 Creating config files for misalignments and steering solutions.ipynb"

In [1]:
from UTILITY_quickstart import *

In [2]:
importedDefaultSettings = loadConfig("setLattice_configs/2024-10-14_twoBunch_baseline.yml")
#importedDefaultSettings = loadConfig("setLattice_configs/2024-10-22_oneBunch_baseline2.yml")

#New DL10 centering 
#importedDefaultSettings = importedDefaultSettings | {"centerDL10" : True}

In [3]:
csrTF = False
evalElement = "PENT"


inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"]
bunchCount = importedDefaultSettings["bunchCount"]
tao = initializeTao(
    inputBeamFilePathSuffix = inputBeamFilePathSuffix,
    
    csrTF = csrTF,
    numMacroParticles=1e3,
    scratchPath = "/tmp",
    randomizeFileNames = True
)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using setLattice_configs/defaults.yml
Number of macro particles = 1000.0
Beam created, written to /tmp/beams/activeBeamFile_11825445267615015763.h5, and reinit to tao


In [4]:
setLattice(tao, **importedDefaultSettings)

with open('./other_configs/pinkCurveMisalignments.json', 'r') as f:
    misalignmentConfig = json.load(f)

applyOtherConfig(tao, misalignmentConfig) 



#Optional: Use one-to-one steering for most correctors
with open('./other_configs/2025-06-13_pinkCurveSteering_2024-10-14_twoBunch_baseline.json', 'r') as f:
    importedSteeringConfig = json.load(f)

applyOtherConfig(tao, importedSteeringConfig) 

In [5]:
totalNumElements = len(tao.lat_list("*", "ele.name"))
combinedElementData = [
    {
        "eleII" : eleII,
        "name" : tao.ele_head(eleII)["name"],
        "key" : tao.ele_head(eleII)["key"],
        "s" : tao.ele_head(eleII)["s"]
    }
    for eleII in range(totalNumElements) 
]

#Keep only elements after L0AFEND
combinedElementData = [ele for ele in combinedElementData if ele["s"] > 4.1274477]

allHKickerNames = [ele["name"] for ele in combinedElementData if ele["key"] == "HKicker"]
allVKickerNames = [ele["name"] for ele in combinedElementData if ele["key"] == "VKicker"]



######################################
#Optional: Limit by s

#optimizerKickerSMin = tao.ele_head("ENDDL10")["s"]

optimizerKickerSMin = tao.ele_head(importedSteeringConfig[-1][0])["s"] #Start from the last kicker in the imported steering configuration
optimizerKickerSMax = tao.ele_head("BEGBC20")["s"]

print( optimizerKickerSMin, optimizerKickerSMax ) 

allHKickerNames = [ ele for ele in allHKickerNames if optimizerKickerSMin < tao.ele_head(ele)["s"] < optimizerKickerSMax]
allVKickerNames = [ ele for ele in allVKickerNames if optimizerKickerSMin < tao.ele_head(ele)["s"] < optimizerKickerSMax]

combinedKickerNameList = allHKickerNames + allVKickerNames

879.809569853447 929.537144536861


In [6]:
def objective(params, tao):

    try: 
        configArr = [ [combinedKickerNameList[i], "BL_KICK", params[i]] for i in range(len(params)) ]
    
        applyOtherConfig(tao, configArr)
        
        trackBeam(tao, trackEnd = "BEGBC20", verbose = False)
    
        P = getBeamAtElement(tao, "BEGBC20")
    
        x = np.median(  P.x ) 
        xp = np.median( P["xp"] ) 
    
        y = np.median(  P.y ) 
        yp = np.median( P["yp"] )

        emitX = smallestIntervalImpliedEmittance(P, plane = "x", percentage = 0.90)
        emitY = smallestIntervalImpliedEmittance(P, plane = "y", percentage = 0.90)

        # x = np.mean(  P.x ) 
        # xp = np.mean( P["xp"] ) 
    
        # y = np.mean(  P.y ) 
        # yp = np.mean( P["yp"] )

        # emitX = (P.twiss(plane = "x", fraction = 0.9))["norm_emit_x"]
        # emitY = (P.twiss(plane = "y", fraction = 0.9))["norm_emit_y"]
    



    
        error = np.sqrt( x ** 2 + xp ** 2 + y ** 2 + yp ** 2 + emitX ** 2 + emitY ** 2 )
    
        #Optional: Give small preference to smaller corrector settings
        error = error + 1e-6 * np.linalg.norm(params)
    
        #print(params, error)
        print(error)
    
        return error

    except:
        print("Failed!")

        return 1e20

In [10]:
from scipy.optimize import minimize

initialGuess = [0 for name in combinedKickerNameList]
# initialGuess = [-3.31002480e-05, -2.84018982e-05, -5.97054546e-05, -4.54406430e-06,
#         6.98590175e-06, -7.39066458e-05, -8.55788530e-05, -9.62538270e-05,
#        -7.63787024e-05, -1.66914678e-05,  1.08120897e-06,  3.15776107e-05,
#         6.60208794e-05,  5.64450812e-05,  6.90698899e-05,  5.45431554e-05,
#         5.07600779e-05,  2.82181671e-05,  1.74592927e-05, -4.87327065e-06,
#        -4.18838345e-05, -6.81327379e-06, -3.85366375e-05, -1.79343684e-05,
#        -2.11206562e-05, -1.25414789e-05,  4.63685169e-06,  1.93572205e-05,
#         2.01162190e-05,  1.90624063e-05,  2.17027692e-05, -1.09453324e-06,
#        -2.25638688e-05, -2.51041567e-05, -3.54688526e-06,  1.60764140e-05,
#         2.26195064e-05,  1.67740835e-05,  2.53503180e-06,  4.09390429e-05,
#         2.18297268e-05,  2.16758059e-05,  5.11094335e-06, -3.11492484e-05,
#        -2.35084378e-05, -2.93575176e-05, -6.03166828e-06,  9.42319935e-06,
#         1.78855405e-05,  1.37560263e-06,  2.47901570e-05,  3.02180438e-05,
#         7.19186826e-06, -1.89266255e-05, -9.46737808e-05, -2.72085810e-05,
#        -1.28822921e-05, -1.30342308e-05,  7.86114777e-06,  3.81919413e-05,
#         2.57793374e-05,  1.15597080e-05,  1.48338447e-05,  6.51883996e-06,
#        -6.78231233e-06, -2.82324425e-05, -1.44881329e-05, -1.97377528e-05,
#         4.03176082e-06,  2.02240244e-05,  1.99580594e-05,  5.19631550e-04,
#         2.48647807e-05,  2.76031333e-05,  1.27363410e-05,  6.76315677e-05,
#         7.37207551e-05,  2.64872165e-05, -1.40032493e-05,  1.07626970e-05,
#        -1.10734334e-06, -1.90817559e-05, -5.62550196e-06,  1.57735173e-06,
#         6.72484991e-05,  7.47161896e-06, -7.03645299e-06, -2.80180648e-05,
#        -5.09343199e-05, -4.18546600e-05, -5.02063060e-05, -3.41751952e-05,
#        -2.22699291e-05, -8.01453939e-06,  4.72489378e-05,  2.47263293e-05,
#         1.46112594e-05,  2.31674727e-05,  2.50868618e-05,  1.34859839e-05,
#         6.43473921e-06,  1.33704526e-05, -3.58532527e-06, -2.25686711e-05,
#        -2.29151396e-05, -1.56055226e-05, -1.17070517e-05, -4.85541998e-06,
#         1.41699274e-05,  2.92470044e-05,  2.38659951e-05,  6.69014779e-06,
#        -1.71728479e-06,  4.11036704e-06, -1.96814933e-05, -1.89890705e-05,
#        -2.08500025e-05, -2.38532006e-05, -1.31654921e-05, -1.75733046e-05,
#        -1.96253906e-05,  1.08256434e-05, -9.79322217e-05,  2.34552809e-06,
#         1.92894307e-05,  1.61517330e-05,  8.97560427e-06, -5.93196689e-06,
#        -2.01650392e-05, -1.66284572e-05, -2.05584690e-05,  4.33243374e-06,
#         1.59001482e-05,  2.18219630e-05,  1.20670380e-05,  1.63547757e-05,
#         1.56656403e-05,  1.87600648e-06, -2.12957840e-05, -1.84605920e-05,
#        -1.97799648e-05, -6.14692612e-06, -1.25677277e-04,  1.20978648e-05,
#         6.66106135e-06,  1.83913559e-05,  2.54254896e-05,  7.06227954e-06,
#        -7.55628769e-06, -9.39024043e-06, -1.48885290e-05, -6.80390135e-06,
#        -1.82948225e-05, -4.10076220e-06, -2.42440283e-06]



bounds = [(-0.02, 0.02) for ele in combinedKickerNameList]


# Perform optimization using Nelder-Mead
result = minimize(
    objective, 
    initialGuess, 
    method='Nelder-Mead',
    #method = "COBYQA",
    #method = "L-BFGS-B", 
    bounds = bounds,
    args = (tao, ),
    options = { "maxiter" : 10000 }
)

print("Optimization Results:")
print(f"Optimal Parameters: {result.x}")
print(f"Objective Function Value at Optimal Parameters: {result.fun}")
print(f"Number of Iterations: {result.nit}")
print(f"Converged: {result.success}")


0.004969835642978552
0.00482395563419092
0.0048855646318606265
0.0048804992074310246
0.004933790692262029
0.005023721064553031
0.005048765546278347
0.005006258810673548
0.00499725348192375
0.004830518362687767
0.00481022400815914
0.0047021257159132615
0.004752304379260617
0.004696427044772807
0.004546998866142663
0.004621011788862917
0.004574206339851212
0.00454875852464587
0.004468734182911338
0.004263273074216669
0.004376109056845237
0.004270572724544984
0.004225027949130614
0.003963017077922233
0.004086976200506799
0.004038511896424291
0.00394968433912896
0.0036406347246415683
0.003749980584820108
0.0035513272866414466
0.0030584225417230057
0.0033962402073791464
0.0032734483204341597
0.003040416306960083
0.002434481998991187
0.002812000905063026
0.002546680130406859
0.0022836271258726186
0.001508721776775658
0.0019246166242302533
0.0016081619391897005
0.0014045290164015165
0.0006425650319627147
0.0009550549376144995
0.0007675773906749982
0.000670941336489473
0.0008831755188189964
0.

In [11]:
result.x

array([ 0.00349527,  0.00071943,  0.00251472,  0.00171953, -0.00901943,
        0.00041927,  0.00017139,  0.00158276])

In [12]:
# configArr = [ [combinedKickerNameList[i], "BL_KICK", result.x[i]] for i in range(len(result.x)) ]

# with open('./other_configs/2025-06-13_pinkCurveSteering_2024-10-14_twoBunch_baseline_supplement.json', 'w') as f:
#     json.dump(configArr, f)
